In [10]:
import pandas as pd
import numpy as np
import math
df = pd.read_excel('/content/DATASET.xlsx')


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [11]:
import os
path = './sections/'
isExist = os.path.exists(path)
if not isExist:
  os.makedirs(path)

In [12]:
df2 = pd.DataFrame()
sec_no = ""
i = 0
new_df = pd.DataFrame()
while i < len(df):
    j = 0
    if type(df.iloc[i][0])==str:
        sec_no = df.iloc[i][0]
        j+=1
    j+=i
    exist = False
    while j < len(df) and type(df.iloc[j][0])!=str:
        if df.iloc[j][1] == 10:
            exist = True
        j += 1
    
    if(exist):
        df2 = df2.append(df.loc[i:j-1,:])
        new_df = new_df.append(df.loc[i:j-1,:])
        new_df.to_csv('./sections/'+sec_no+'.csv',index=False)
        new_df = new_df.iloc[0:0]
    i = j

df2.reset_index(inplace=True,drop=True)


In [13]:
test = df2[df2.year == 10]
test = test.reset_index()
test.drop(columns=['index', 'Unnamed: 0'], inplace=True)
test

,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,10,324.0,53.6,19.0,0.0,2,105,7567,1885,16.8,105.0,65.0,9,1.960
1,10,327.7,62.0,22.7,0.0,2,102,30525,10234,76.5,44.3,5.0,9,0.795
2,10,487.7,93.2,22.5,0.0,4,110,2140,1270,78.5,0.0,0.0,16,1.432
3,10,543.6,93.2,22.5,0.0,4,110,2140,1270,28.5,60.5,67.0,11,2.004
4,10,434.4,149.0,11.7,72.0,3,110,13142,6225,0.0,276.2,34.0,6,1.792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,514.0,1226.4,10.1,203.6,16,114,7672,303,219.7,206.2,10.0,8,1.607
96,10,901.0,1531.0,11.1,34.9,40,100,15150,1077,5.9,74.5,33.0,13,1.087
97,10,1181.0,1364.9,9.2,274.3,24,112,12500,880,131.4,72.4,18.0,12,0.996
98,10,662.0,1110.7,12.8,33.2,27,117,3324,1356,193.7,305.6,180.0,12,2.212


In [14]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error

In [15]:
import glob
path = "./sections/*.csv"
for fname in glob.glob(path):
    tdf = pd.read_csv(fname)
    tdf.drop(columns=['Unnamed: 0'],inplace=True)
    sec_no = fname.split('/')[2].split('.')[0]

    rmse_df = pd.DataFrame(columns=['Para-9', 'Para-10',	'Para-11',	'Para-12',	'Para-13'])

    for i in range(9,14):
        folder_path = './rmse comparisons/'
        isExist = os.path.exists(folder_path)
        if not isExist:
            os.makedirs(folder_path)

        x = tdf.drop(columns=['Para-9', 'Para-10',	'Para-11',	'Para-12',	'Para-13'], axis=1)
        y = tdf['Para-'+str(i)]
        x_train, x_test = x.iloc[:-1, :], x.iloc[-1:, :]
        y_train, y_test = y.iloc[:-1], y.iloc[-1:]
        model = LGBMRegressor(objective="regression",metric="rmse",boosting_type='dart')
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        rmse = np.round(math.sqrt(mean_squared_error(y_test, predictions)), 3)
        actual = y_test.values
        rmse_df.loc[0, 'Para-'+str(i)] = actual[0]
        rmse_df.loc[1, 'Para-'+str(i)] = np.round(predictions[0], 3)
        rmse_df.loc[2, 'Para-'+str(i)] = rmse
    
    rmse_df.to_csv(folder_path+'/'+sec_no+'.csv', index=False)

In [16]:
rmse_collection = []

folder_path = './graphs/'
isExist = os.path.exists(folder_path)
if not isExist:
  os.makedirs(folder_path)

path = "./rmse comparisons/*.csv"
for fname in glob.glob(path):
  rmse_df = pd.read_csv(fname)
  sec_no = fname.split('/')[2].split('.')[0]

  mean_rmse = np.round(rmse_df.iloc[2, :].mean(axis=0), 3)
  rmse_collection.append(mean_rmse)
  line_1 = rmse_df.iloc[0, :]
  line_2 = rmse_df.iloc[1, :]
  fig, ax = plt.subplots()
  ax.plot(line_1, color = 'green', label = 'Actual Data')
  ax.plot(line_2, color = 'red', label = 'Predicted Data')
  plt.title("RMSE Graph: "+str(mean_rmse))
  ax.legend()
  plt.savefig("./graphs/"+str(sec_no)+".png")
  plt.close()

In [17]:
f_rmse = sum(rmse_collection)/len(rmse_collection)
print("RMSE: ", np.round(f_rmse, 3))

RMSE:  34.091
